<a href="https://colab.research.google.com/github/BrunaFerreira/Coursera_AI_for_Medical_Diagnosis/blob/master/Lab2_Arquitetura_Densenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Aluna : Bruna Ferreira

# Densenet
* AI for Medicine Course 1 Week 1 lecture exercises
* AI4M_C1_W1_lecture_ex_03


## Modelo Densenet pré-treinado para classificação de imagens

<b>Densenet</b> é uma rede convolucional em que cada camada está conectada a todas as outras camadas mais profundas da rede
- A primeira camada está conectada à 2ª, 3ª, 4ª, etc.
- A segunda camada está conectada à 3ª, 4ª, 5ª, etc.

Como isso:

<img src = "https://pytorch.org/assets/images/densenet1.png" alt = "U-net Image" width = "400" align = "middle" />

Explicação detalhada de Densenet, verifique o artigo de Gao Huang et al. 2018 chamado  <a href = 'https://arxiv.org/pdf/1608.06993.pdf'>Densely Connected Convolutional Network</a>.

In [2]:
#Import DenseNet from Keras
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K

Será carregado um conjunto de pesos pré-treinados para reduzir o tempo de treinamento.

In [3]:
data_pre_trained = '/content/drive/MyDrive/0_Cursos/1. Coursera/AI Medicine /Code/data/images/nih/densenet.hdf5'

In [4]:
# Criando a base pré-treinada do modelo
base_model = DenseNet121( include_top=False)
base_model.load_weights(data_pre_trained, by_name=True)

In [5]:
# Mostrando Detalhes do Modelo
base_model.summary()

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
________________________________________________________________________________________

#### Cinco primeiras camadas (Layers)

In [6]:
layers_l = base_model.layers

print(" Cinco Primeiras Camadas")
layers_l[0:5]

 Cinco Primeiras Camadas


#### Cinco últimas camadas (Layers)

In [7]:
layers_l[-6:-1]

#### Camadas Convolucionais 
* Cinco primeiras

In [8]:
# Get the convolutional layers and print the first 5
conv2D_layers = [layer for layer in base_model.layers 
                if str(type(layer)).find('Conv2D') > -1]
conv2D_layers[0:5]

* Quantidade de camadas convolucionais

In [9]:
print(f"Há {len(conv2D_layers)} camadas convolucioanis.")

Há 120 camadas convolucioanis.


#### Channels - Canais 
* 3 Canais na entrada

In [10]:
base_model.input

<KerasTensor: shape=(None, None, None, 3) dtype=float32 (created by layer 'input_1')>

* 1024 Canais na saída

In [11]:
x = base_model.output
x

<KerasTensor: shape=(None, None, None, 1024) dtype=float32 (created by layer 'relu')>

#### Definição de cinco rótulos de classe

In [12]:
labels = ['Emphysema', 
          'Hernia', 
          'Mass', 
          'Pneumonia',  
          'Edema']
n_classes = len(labels)
print(f"Classes a serem identificadas por esse modelo: {n_classes} .")

Classes a serem identificadas por esse modelo: 5 .


#### Adicionar camada 
* <i>global spatial average pooling layer - </i>Camada de agrupamento de média espacial global

In [13]:
x_pool = GlobalAveragePooling2D()(x)
x_pool

<KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'global_average_pooling2d')>

* <i> logistic layer - </i> camada logística do mesmo tamanho que o número de classes

In [14]:
predictions = Dense(n_classes, activation="sigmoid")(x_pool)
print(f"Predictions tem {n_classes} unidades, uma para cada classe")
predictions

Predictions tem 5 unidades, uma para cada classe


<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'dense')>

### Atualização do modelo

In [15]:
model = Model(inputs=base_model.input, outputs=predictions)

### Compilando Modelo

In [16]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy')